In [113]:
r = [[[''], ['', '', '', 'Revogado', 'INFOTEMP 0916/2018', '', ''], ['', 'COMPULSORIO CTC COM A RDO SINOP PARA ACFT OPR NA FIZ SINOP ', '', '', ''], ['', '', 'Duração:', ' ', '04/10/18 00:00 a 31/12/19 23:59 UTC', ' ', ' Divulgação: ', '01/10/2018 13:55', '', ''], ['', '']], [[''], ['', '', '', 'Revogado', 'INFOTEMP 0890/2018', '', ''], ['', 'ACFT EM OPS DE POUSO E DECOLAGEM DEVERAO EFETUAR CTC COM A RDO SINOP NA FREQ 130.175MHZ ', '', '', ''], ['', '', 'Duração:', ' ', '27/09/18 18:05 a 31/12/19 23:59 UTC', ' ', ' Divulgação: ', '27/09/2018 20:19', '', ''], ['', '']]]

[list(filter(str.strip, value)) for value in r[0]]

[[],
 ['Revogado', 'INFOTEMP 0916/2018'],
 ['COMPULSORIO CTC COM A RDO SINOP PARA ACFT OPR NA FIZ SINOP '],
 ['Duração:',
  '04/10/18 00:00 a 31/12/19 23:59 UTC',
  ' Divulgação: ',
  '01/10/2018 13:55'],
 []]

# Estudo de oportunidades para PBZPA

In [1]:
# helper function
from os import listdir
def dir_files(name: str, folder: str = None):
    if folder:
        files = [folder + e for e in listdir(folder) if (e[-4:] == ".xls") or (e[-4:] == ".csv")]
    else:
        files = [e for e in listdir() if (e[-4:] == ".xls") or (e[-4:] == ".csv")]
    print(f"{name}: {', '.join(files)}")
    return files

In [4]:
from pandas import read_excel
from pandas import read_csv

public_private = dir_files('Aeródromos públicos e privados', r'E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/')
aerodromos_publicos =  read_excel(public_private[1], sheet_name=0, header=[1,2])
aerodromos_privados = read_excel(public_private[0], sheet_name=0, header=[0,1])

print('\n')

aisweb = dir_files('AISWeb')
aisweb =  read_csv(aisweb[0], sep=',', encoding='latin-1')

aisweb.insert(loc=4, column="Categoria",value='')
icao_aeropublicos = aerodromos_publicos.iloc[:,0].values
icao_aeroprivados = aerodromos_privados.iloc[:,0].values
for i, icao in enumerate(aisweb.ICAO):
    if icao in icao_aeropublicos:
        aisweb.at[i, 'Categoria'] = 'Público'
    elif icao in icao_aeroprivados:
        aisweb.at[i, 'Categoria'] = 'Privado'


    value[]
    aisweb.at[i, 'NOTAM'] = value

Aeródromos públicos e privados: E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/Aeródromos Privados_28MAR2019.xls, E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/Aeródromos Públicos_28MAR2019.xls


AISWeb: output_aisweb_4428c205cd.csv


In [30]:
aisweb[200:210]

,timestamp,ICAO,CIAD,Aeródromo,Categoria,Cidade,UF,Status,NOTAM,Debug
200,2019-04-18T04:11:25.486319-03:00,SNXY,PA0113,Cikel Brasil Verde,Privado,PORTEL,PA,OK,[[[]]],{'response_status_code': 200}
201,2019-04-18T04:11:29.600286-03:00,SIVJ,MG0112,Cirrus Sociedade Aerodesportiva,Privado,JABOTICATUBAS,MG,OK,[[[]]],{'response_status_code': 200}
202,2019-04-18T04:11:34.210320-03:00,SJAC,PA0065,Citropar,Privado,CAPITÃO POÇO,PA,OK,[[[]]],{'response_status_code': 200}
203,2019-04-18T04:11:38.808320-03:00,SJEH,SP0206,Clemente Verillo,Privado,RIBEIRÃO BONITO,SP,OK,[[[]]],{'response_status_code': 200}
204,2019-04-18T04:11:43.656322-03:00,SIVG,MT0141,Clube Aerodesportivo Selva,Privado,SINOP,MT,OK,"[[['Revogado INFOTEMP 0916/2018], [COMPULSORIO...",{'response_status_code': 200}
205,2019-04-18T04:11:48.384320-03:00,SIAN,RJ0020,Clube CEU,Privado,RIO DE JANEIRO,RJ,OK,"[[['D0489/19 R D0488/19 10/04/2019 18:17], [PJ...",{'response_status_code': 200}
206,2019-04-18T04:11:52.344359-03:00,SWON,SC0041,Clube de Aviação Céu Azul,Privado,ARAQUARI,SC,OK,[[[]]],{'response_status_code': 200}
207,2019-04-18T04:11:56.781324-03:00,SJXA,MG0137,Clube de Aviação de Itaúna,Privado,ITAÚNA,MG,OK,"[[['Revogado INFOTEMP 0862/2018], [AD INSERIR ...",{'response_status_code': 200}
208,2019-04-18T04:12:00.929359-03:00,SWES,PR0095,Clube de Aviação Experimental do Paraná,Privado,IBIPORÃ,PR,OK,[[[]]],{'response_status_code': 200}
209,2019-04-18T04:12:05.173323-03:00,SWEQ,PA0139,Clube de Esportes Aéreos e Náuticos do Pará,Privado,BELÉM,PA,OK,[[[]]],{'response_status_code': 200}


In [27]:
aisweb.NOTAM.iloc[100]

[["['INFOTEMP 1492/2018"],
 ['AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO RMK: EXC ACFT OPS EM CARATER EMERGENCIAL E PRECARIO, SEMPRE NO INTERESSE DO ATENDIMENTO DOS POVOS INDIGENAS E APENAS POR ENTES A BEM DOS INTERESSES DO SERVICO PUBLICO'],
 ['Duração: 14/12/18 12:08 a 31/12/20 23:59 UTC',
  "Divulgação: 14/12/2018 12:41', 'Revogado INFOTEMP 1186/2018"],
 ['AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO RMK: EXC ACFT DESTINADAS APOIAR A SECRETARIA ESPECIAL DE SAUDE INDIGENA E CASO O AD ESTEJA LOCALIZADO NA AREA RESTRITA SBR 701 (FRONTEIRA), A ACFT ENVOLVIDA NA MISSAO DEVERA POSSUIR OBRIGATORIAMENTE UMA AVOAR VALIDA EMITIDA PELO EXMO SR COMANDANTE DA ALA 8'],
 ['Duração: 09/11/18 18:39 a 31/12/20 23:59 UTC ',
  " Divulgação: 09/11/2018 18:39']"]]

In [5]:
aisweb.count()

timestamp    2630
ICAO         2630
CIAD         2569
Aeródromo    2568
Categoria    2630
Cidade       2568
UF           2568
Status       2630
NOTAM        2630
Debug        2630
dtype: int64

### Há valores nulos?

In [6]:
aisweb.isnull().sum()

timestamp     0
ICAO          0
CIAD         61
Aeródromo    62
Categoria     0
Cidade       62
UF           62
Status        0
NOTAM         0
Debug         0
dtype: int64

### Aeródromos não encontrados (fechado, localidade cancelada, Localidade reservada aguardando portaria da ANAC)

#### Aeródromos que constam na lista da ANAC, mas não foram encontrados no AISWeb

In [8]:
aero_naoencontrado = aisweb[aisweb.Status == "O aeródromo não foi encontrado"]
print(f"Total de aeródromos com Status: {aero_naoencontrado.ICAO.count()}")

Total de aeródromos com Status: 3


#### Localidade cancelada

In [9]:
aero_localizacao_cancelada = aisweb[aisweb.Status == "Localidade cancelada"]
print(f"Total de aeródromos ({aero_localizacao_cancelada.Status[:1].item()}): {aero_localizacao_cancelada.ICAO.count()}")

Total de aeródromos (Localidade cancelada): 3


#### Aeródromos com localidade reservada aguardando portaria da ANAC

In [10]:
aero_localizacao_reservada = aisweb[aisweb.Status == "Localidade reservada. Aguardando portaria da ANAC."]
print(f"Total de aeródromos ({aero_localizacao_reservada.Status[:1].item()}): {aero_localizacao_reservada.ICAO.count()}")

Total de aeródromos (Localidade reservada. Aguardando portaria da ANAC.): 54


#### Aeródromos fechados até determinada data

In [11]:
filtro = "Aeródromo FECHADO até"
aero_fechado = aisweb[aisweb.Status.str.contains(filtro) == True]
print(f"Total de aeródromos fechados: {aero_fechado.ICAO.count()}")

Total de aeródromos fechados: 165


#### Aeródromos com dados faltando
Estão iniciando atividade? ICAO consta no AISWeb, não foi exibida nenhuma mensagem de Status.

In [12]:
aero_null = aisweb[aisweb.Cidade.isnull()]
print(f"Total de aeródromos com valores nulos: {aero_null.ICAO.count()}")

Total de aeródromos com valores nulos: 62


In [13]:
aero_null.isnull().sum()

timestamp     0
ICAO          0
CIAD         61
Aeródromo    62
Categoria     0
Cidade       62
UF           62
Status        0
NOTAM         0
Debug         0
dtype: int64

In [14]:
aero_null.head()

,timestamp,ICAO,CIAD,Aeródromo,Categoria,Cidade,UF,Status,NOTAM,Debug
572,2019-04-18T04:10:20.346286-03:00,SSXI,NaN,NaN,Privado,NaN,NaN,Localidade cancelada,[[[]]],{'response_status_code': 200}
776,2019-04-18T04:13:15.719320-03:00,SJWH,NaN,NaN,Privado,NaN,NaN,Localidade não encontrada,[[[]]],{'response_status_code': 200}
916,2019-04-18T04:15:42.500285-03:00,SITM,NaN,NaN,Privado,NaN,NaN,Localidade cancelada,[[[]]],{'response_status_code': 200}
1160,2019-04-18T04:13:19.136320-03:00,SJZR,NaN,NaN,Privado,NaN,NaN,Localidade reservada. Aguardando portaria da A...,[[[]]],{'response_status_code': 200}
1166,2019-04-18T04:13:42.893359-03:00,SDYG,NaN,NaN,Privado,NaN,NaN,Localidade reservada. Aguardando portaria da A...,[[[]]],{'response_status_code': 200}


## NOTAM

#### Aeródromos contendo a string "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO"

In [28]:
filtro = "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO"
ad_clsd_pbzpa = aisweb[aisweb.NOTAM.str.contains(filtro)]
print(f"Total de aeródromos com \"AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO\": {ad_clsd_pbzpa.ICAO.count()}")

KeyError: "None of [Float64Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              ...\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n             dtype='float64', length=2630)] are in the [columns]"

#### Com fechamento não revogado ou está agendado para fechamento

In [ ]:
filtro = ".*Revogado INFOTEMP [0-9]*/[0-9]*.{0,6}AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO.*"
oportunidades = ad_clsd_pbzpa[~ad_clsd_pbzpa.NOTAM.str.contains(filtro, regex=True) & ad_clsd_pbzpa.Status.str.contains('Aeródromo FECHADO')]
print(f"Total de prospects para novos negócios: {oportunidades.ICAO.count()}")

#### Revogado o fechamento

In [ ]:
filtro = ".*Revogado INFOTEMP [0-9]*/[0-9]*.{0,6}AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO.*"
ad_clsd_pbzpa_rev = ad_clsd_pbzpa[ad_clsd_pbzpa.NOTAM.str.contains(filtro, regex=True)]
print(f"Total de aeródromos com \"AD CLSD\" revogado: {ad_clsd_pbzpa_rev.ICAO.count()}")

#### Com fechamento revogado, mas somente em missões de caráter emergencial
ou seja, candidatos para INFRACEA | Social. Da retorno de mídia. Será que há empresas com interesse na facilitação de acesso à região? Será há benefício fiscal para empresas que financiarem o produto PBZPA da INFRACEA nestas regiões?

In [ ]:
filtro = "CARATER EMERGENCIAL"
ad_clsd_pbzpa_rev_emg = ad_clsd_pbzpa_rev[ad_clsd_pbzpa_rev.NOTAM.str.contains(filtro)]
print(f"Total de aeródromos com \"AD CLSD\" revogado em caráter emergencial: {ad_clsd_pbzpa_rev_emg.ICAO.count()}")

#### Revogação do fechamento ratificada
situação inferida: gestão fraca? Estão desesperados esperando a INFRACEA | Aeroportos?

In [ ]:
ad_clsd_pbzpa_rev_ratif = ad_clsd_pbzpa_rev[ad_clsd_pbzpa_rev.ICAO.isin(aero_fechado.ICAO)]
print(f"Total de aeródromos com funcionamento turbulento: {ad_clsd_pbzpa_rev_ratif.ICAO.count()}")

##### Exportar para excel

In [ ]:
#ad_clsd_pbzpa.to_excel('notam_ad_clsd_pbzpa.xlsx', index=False, na_rep='')